# LDA Topic Modelling in Python: Example 2, Part 2

This notebook is a use case of Gensim’s LDA model based on the [GitHub Topic Modelling Textbook](https://github.com/wjbmattingly/topic_modeling_textbook/blob/main/03_03_lda_model_demo_bigrams_trigrams.ipynb) posted by wjbmattingly.

## Importing the Required Libraries

In [1]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
# import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/Users/anastasiya.kuzmich/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

## Preparing the data

In [3]:
stopwords = stopwords.words("english")
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
data = load_data("ushmm_dn.json")["texts"]

print (data[0][0:90])

 My name David Kochalski. I was born in a small town called , and I was born May 5, 1928. 


### Lemmatisation

In [5]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]): # these refer to their spacy tags
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

name bear small town call bear very hard work child father mother small mill flour buckwhe


### Simple pre-processing

In [6]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['name', 'bear', 'small', 'town', 'call', 'bear', 'very', 'hard', 'work', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go', 'school']


### Getting bigrams & trigrams

In [7]:
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print (data_bigrams_trigrams[0][0:20])

['name', 'bear', 'small', 'town', 'call', 'bear', 'very', 'hard', 'work', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go', 'school']


### TF-IDF Removal

In [8]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

## Modelling

In [10]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [11]:
test_doc = corpus[-1]

vector = lda_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(0, 0.01436125), (1, 0.33370167), (2, 0.05914124), (3, 0.037807904), (5, 0.023313878), (6, 0.041571956), (7, 0.42860782), (8, 0.011850493), (9, 0.049591985)]
[(7, 0.42860782), (1, 0.33370167), (2, 0.05914124), (9, 0.049591985), (6, 0.041571956), (3, 0.037807904), (5, 0.023313878), (0, 0.01436125), (8, 0.011850493)]


In [14]:
lda_model.save("models/test_model.model")

In [15]:
new_model = gensim.models.ldamodel.LdaModel.load("models/test_model.model")

In [16]:
test_doc = corpus[-1]

vector = new_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(0, 0.014361267), (1, 0.33370167), (2, 0.0591419), (3, 0.037807915), (5, 0.023313805), (6, 0.04157201), (7, 0.42860773), (8, 0.011849773), (9, 0.049592175)]
[(7, 0.42860773), (1, 0.33370167), (2, 0.0591419), (9, 0.049592175), (6, 0.04157201), (3, 0.037807915), (5, 0.023313805), (0, 0.014361267), (8, 0.011849773)]


## Visualise the data

In [18]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=30)

/Users/anastasiya.kuzmich/Library/Python/3.9/lib/python/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/site-packages/past/builtins/misc.py:45:

In [20]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.341086 -0.064883       1        1  20.502022
2      0.070877  0.045932       2        1  20.415466
7     -0.251273  0.257057       3        1  13.946938
3      0.240959 -0.233042       4        1  12.548471
9      0.275392 -0.015886       5        1   8.708429
5      0.262957  0.202393       6        1   6.734442
8      0.051918  0.310552       7        1   6.540354
0      0.039664 -0.304159       8        1   5.338826
6     -0.181935  0.057746       9        1   3.129913
4     -0.167472 -0.255710      10        1   2.135140, topic_info=         Term         Freq        Total Category  logprob  loglift
337    ghetto  2228.000000  2228.000000  Default  30.0000  30.0000
663  remember  1093.000000  1093.000000  Default  29.0000  29.0000
852       use  1679.000000  1679.000000  Default  28.0000  28.0000
187    course  1384.000000  1384.000000  Default  27.0000  27.0000
111   brother   830.000000   830.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
98       bomb    19.499338   187.386363  Topic10  -5.3492   1.5838
440      kill    28.495086  1298.676667  Topic10  -4.9699   0.0273
439       kid    21.258835   384.496840  Topic10  -5.2628   0.9515
115  building    17.361449   353.555944  Topic10  -5.4653   0.8328
350     great    16.299951   255.037627  Topic10  -5.5284   1.0964

[708 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.283483     able
0         3  0.004200     able
0         5  0.657261     able
0         7  0.002100     able
0         8  0.052497     able
...     ...       ...      ...
2551      1  0.016236  zionist
2551      3  0.032472  zionist
2551      7  0.032472  zionist
2551      8  0.097415  zionist
2551      9  0.811789  zionist

[1517 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 8, 4, 10, 6, 9, 1, 7, 5])